<a href="https://colab.research.google.com/github/justingrammens/machine_learning/blob/master/v3_ROME_Accumulator_ROMEStyle_YAML_ExistingTBs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROME-style RTL Feedback Loop — Accumulator (SRA)

This notebook follows the structure/pattern of `ROME_Demo_HDHP_11_6_2025.ipynb`, adapted for the Accumulator project:

- **Specs**: `/content/accum_module_specs.yaml`
- **Existing testbenches**: `/content/<module>tb.v` (no TB generation)
- Runs **each sub-module**, then builds/verifies **top-level** `accum_core` last.
- Leaf modules compile **only their own RTL**; top-level compiles **all generated RTL**.

**Failure policy:**
- If a **leaf module fails**, the notebook **continues** to the next leaf module.
- If **any** leaf modules fail, the notebook **does not attempt** to compile/run the top-level module.

Expected files in Colab `/content`:
- `accum_module_specs.yaml`
- `accum_enable_edgetb.v`, `accum_signal_bus_iftb.v`, …, `accum_coretb.v`


In [ ]:
#@title Setting up the notebook

!pip -q install openai anthropic pyyaml
!apt-get update -qq
!apt-get install -y -qq iverilog


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 14.4 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package iverilog.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../iverilog_11.0-1.1_amd64.deb ...
Unpacking iverilog (11.0-1.1) ...
Setting up iverilog (11.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
#@title Select Model

# OpenAI example: GPT-5.2 (replace with the model id available to your account if needed)
model_choice = "gpt-5.2"

# model_choice = "claude-3-7-sonnet-20250219"

# Provider wrapper used by generate_verilog()
# "ChatGPT" -> OpenAI (Colab secret 'ROME-Colab' or env OPENAI_API_KEY)
# "Claude"  -> Anthropic (env CLAUDE_API_KEY)
import os
os.environ["MODEL"] = "ChatGPT"


In [ ]:
#@title Utility functions (ROME-style)

import os, re, time, subprocess
from pathlib import Path
import yaml

import openai
import anthropic

try:
    from google.colab import userdata
except Exception:
    userdata = None

class Conversation:
    def __init__(self, log_file=None):
        self.messages = []
        self.log_file = log_file
        if self.log_file:
            Path(self.log_file).write_text("")

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})
        if self.log_file:
            with open(self.log_file, "a") as f:
                f.write(f"\n[{role.upper()}]\n{content}\n")

class ChatGPT:
    def __init__(self):
        api_key = None
        if userdata is not None:
            try:
                api_key = userdata.get("ROME-Colab")
            except Exception:
                api_key = None
        api_key = api_key or os.environ.get("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("Missing OpenAI API key. Set Colab secret 'ROME-Colab' or env OPENAI_API_KEY.")
        self.client = openai.OpenAI(api_key=api_key)

    def generate(self, conv, model_id=""):
        resp = self.client.chat.completions.create(
            model=model_id,
            messages=conv.messages
        )
        return resp.choices[0].message.content

class Claude:
    def __init__(self):
        api_key = os.environ.get("CLAUDE_API_KEY")
        if not api_key:
            raise ValueError("Missing CLAUDE_API_KEY.")
        self.client = anthropic.Anthropic(api_key=api_key)

    def generate(self, conv, model_id=""):
        system_msgs = [m["content"] for m in conv.messages if m["role"] == "system"]
        system = system_msgs[-1] if system_msgs else ""
        user_msgs = [{"role": "user", "content": m["content"]} for m in conv.messages if m["role"] != "system"]
        msg = self.client.messages.create(
            model=model_id,
            system=system,
            messages=user_msgs,
            max_tokens=4096
        )
        out = []
        for b in msg.content:
            if getattr(b, "type", None) == "text":
                out.append(b.text)
        return "\n".join(out).strip()

def generate_verilog(conv, model_type, model_id=""):
    if model_type == "ChatGPT":
        model = ChatGPT()
    elif model_type == "Claude":
        model = Claude()
    else:
        raise ValueError("Invalid model type")
    return model.generate(conv, model_id=model_id)

def extract_named_module(text, module_name):
    text = re.sub(r"```.*?\n", "", text, flags=re.DOTALL).replace("```", "")
    pat = rf"\bmodule\s+{re.escape(module_name)}\b.*?\bendmodule\b"
    m = re.search(pat, text, flags=re.DOTALL)
    if m:
        return m.group(0).strip() + "\n"
    any_mod = re.search(r"\bmodule\b.*?\bendmodule\b", text, flags=re.DOTALL)
    return (any_mod.group(0).strip() + "\n") if any_mod else (text.strip() + "\n")

def write_code_to_file(code, out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_text(code)

def load_specs(spec_path: Path):
    spec = yaml.safe_load(spec_path.read_text())
    mods_by_name = {m["name"]: m for m in spec["modules"]}
    order = list(spec.get("module_generation_order", list(mods_by_name.keys())))
    return spec, mods_by_name, order

def ports_to_ansi_list(ports_dict):
    items = []
    for pname, pdir in ports_dict.items():
        pdir = str(pdir).split("#")[0].strip()
        items.append(f"{pdir} {pname}")
    return ", ".join(items)

def spec_prompt_for_module(mod_spec):
    name = mod_spec["name"]
    intent = mod_spec.get("intent", "")
    ports = "\n".join([f"- {str(v).split('#')[0].strip()} {k}" for k,v in mod_spec["ports"].items()])
    behavior = "\n".join([f"- {b}" for b in mod_spec.get("behavior", [])])
    notes = mod_spec.get("notes", [])
    notes_txt = "\n".join([f"- {n}" for n in notes]) if notes else "(none)"
    return f"""// MODULE SPECIFICATION: {name}
// INTENT: {intent}
//
// PORTS:
// {ports}
//
// BEHAVIOR:
// {behavior}
//
// NOTES:
// {notes_txt}
"""

def iverilog_build(out_exe: Path, rtl_files, tb_file: Path, inc_dirs=None):
    inc_dirs = inc_dirs or []
    cmd = ["iverilog", "-g2012", "-o", str(out_exe)]
    for d in inc_dirs:
        cmd += ["-I", str(d)]
    cmd += [str(p) for p in rtl_files] + [str(tb_file)]
    return subprocess.run(cmd, capture_output=True, text=True)

def vvp_run(exe: Path):
    return subprocess.run(["vvp", str(exe)], capture_output=True, text=True)


In [ ]:
#@title Feedback Loop (uses existing TBs)

def verilog_loop(design_prompt,
                 module_name,
                 rtl_out_path: Path,
                 tb_path: Path,
                 rtl_files_to_compile,
                 max_iterations,
                 model_type,
                 model_id,
                 log_path: Path,
                 build_dir: Path):

    conv = Conversation(log_file=str(log_path))
    conv.add_message("system",
        "You are an autocomplete engine for Verilog code. "
        "Given a Verilog module specification, provide a completed synthesizable Verilog/SystemVerilog module. "
        "Do not generate explanations. Do not generate test benches. "
        "Return ONLY code for the requested module. "
        "No delays, no force/release. Use nonblocking assignments for sequential logic."
    )
    conv.add_message("user", design_prompt)

    last_rtl = ""
    total_start = time.time()
    gen_time = 0.0
    err_time = 0.0

    for it in range(1, max_iterations + 1):
        print(f"\n[{module_name}] iteration {it}/{max_iterations}")

        t0 = time.time()
        response = generate_verilog(conv, model_type=model_type, model_id=model_id)
        gen_time += (time.time() - t0)

        rtl = extract_named_module(response, module_name)
        last_rtl = rtl
        write_code_to_file(rtl, rtl_out_path)

        exe = build_dir / f"sim_{module_name}.out"
        proc = iverilog_build(exe, rtl_files_to_compile, tb_path, inc_dirs=[tb_path.parent, rtl_out_path.parent])

        if proc.returncode != 0:
            err_time_start = time.time()
            msg = (
                "The RTL or testbench failed to compile.\n"
                "iverilog stderr:\n" + proc.stderr + "\n"
                "iverilog stdout:\n" + proc.stdout + "\n"
                "Please correct ONLY the RTL for module " + module_name + " and output the full corrected module."
            )
            print("Compile failed.")
            conv.add_message("assistant", response)
            conv.add_message("user", msg + "\n\nCurrent RTL:\n" + last_rtl)
            err_time += (time.time() - err_time_start)
            continue

        run = vvp_run(exe)
        sim_out = (run.stdout or "") + (run.stderr or "")

        if run.returncode != 0 or ("passed!" not in sim_out):
            err_time_start = time.time()
            msg = (
                "The testbench did not pass.\n"
                "Simulation output:\n" + sim_out + "\n"
                "Please correct ONLY the RTL for module " + module_name + " and output the full corrected module."
            )
            print("Simulation failed or did not pass.")
            conv.add_message("assistant", response)
            conv.add_message("user", msg + "\n\nCurrent RTL:\n" + last_rtl)
            err_time += (time.time() - err_time_start)
            continue

        print(f"{module_name}: passed!")
        total_time = time.time() - total_start
        return total_time, gen_time, err_time, True

    total_time = time.time() - total_start
    return total_time, gen_time, err_time, False


In [ ]:
#@title Hierarchical Loop (YAML → leaf modules → top-level) with non-stop leaf failures

def hier_gen_from_yaml(spec_path: Path,
                       tb_dir: Path,
                       out_root: Path,
                       top_module: str = "accum_core",
                       max_iterations: int = 10):

    spec, mods_by_name, yaml_order = load_specs(spec_path)

    def tb_for(m): return tb_dir / f"{m}tb.v"
    modules_with_tb = [m for m in yaml_order if tb_for(m).exists()]

    leaf = [m for m in modules_with_tb if m != top_module]
    have_top_tb = top_module in modules_with_tb

    print("Leaf modules:")
    for m in leaf:
        print(" -", m, "->", tb_for(m))
    print("\nTop module:", top_module, "->", tb_for(top_module) if have_top_tb else "(no tb found)")

    model_type = os.environ.get("MODEL", "ChatGPT")

    # Track results without stopping on leaf failures
    results = {}
    times = {}

    generated_paths = {}  # only successful modules will be included here

    # ----------
    # 1) Run ALL leaves
    # ----------
    for m in leaf:
        mod_spec = mods_by_name[m]
        tb_path = tb_for(m)

        mod_dir = out_root / m
        mod_dir.mkdir(parents=True, exist_ok=True)
        rtl_out = mod_dir / f"{m}.v"
        log_path = mod_dir / "log.txt"
        build_dir = mod_dir / "build"
        build_dir.mkdir(parents=True, exist_ok=True)

        prompt = spec_prompt_for_module(mod_spec)
        ansi_ports = ports_to_ansi_list(mod_spec["ports"])
        prompt += f"\n// Please implement:\nmodule {m}({ansi_ports});\n// Insert code here\nendmodule\n"

        # Leaf compiles ONLY its own RTL + TB
        rtl_files = [rtl_out]

        total, gen, err, ok = verilog_loop(
            design_prompt=prompt,
            module_name=m,
            rtl_out_path=rtl_out,
            tb_path=tb_path,
            rtl_files_to_compile=rtl_files,
            max_iterations=max_iterations,
            model_type=model_type,
            model_id=model_choice,
            log_path=log_path,
            build_dir=build_dir
        )

        results[m] = ok
        times[m] = {"total": total, "gen": gen, "err": err}

        if ok:
            generated_paths[m] = rtl_out

    failed_leaf = [m for m in leaf if not results.get(m, False)]

    print("\nLeaf results:")
    for m in leaf:
        print(f" - {m}: {'PASS' if results[m] else 'FAIL'}")

    # ----------
    # 2) Top-level policy: only attempt top if ALL leaves passed and top TB exists
    # ----------
    if failed_leaf:
        print("\nOne or more leaf modules FAILED:")
        for m in failed_leaf:
            print(" -", m)
        print("\nPer policy: skipping top-level build/verification.")
        return {"generated": generated_paths, "results": results, "times": times, "skipped_top": True}

    if not have_top_tb:
        print("\nTop-level TB not found; nothing to run for top-level.")
        return {"generated": generated_paths, "results": results, "times": times, "skipped_top": True}

    # ----------
    # 3) Run top-level
    # ----------
    m = top_module
    mod_spec = mods_by_name[m]
    tb_path = tb_for(m)

    mod_dir = out_root / m
    mod_dir.mkdir(parents=True, exist_ok=True)
    rtl_out = mod_dir / f"{m}.v"
    log_path = mod_dir / "log.txt"
    build_dir = mod_dir / "build"
    build_dir.mkdir(parents=True, exist_ok=True)

    prompt = spec_prompt_for_module(mod_spec)
    prompt += "\n// Previously generated leaf submodules (for reference):\n"
    for sm, p in generated_paths.items():
        prompt += f"\n// ---- {sm} ----\n{p.read_text()}\n"

    ansi_ports = ports_to_ansi_list(mod_spec["ports"])
    prompt += f"\n// Please implement:\nmodule {m}({ansi_ports});\n// Insert code here\nendmodule\n"

    # Top compiles all PASS leaf RTL + top RTL
    rtl_files = [generated_paths[x] for x in leaf] + [rtl_out]

    total, gen, err, ok = verilog_loop(
        design_prompt=prompt,
        module_name=m,
        rtl_out_path=rtl_out,
        tb_path=tb_path,
        rtl_files_to_compile=rtl_files,
        max_iterations=max_iterations,
        model_type=model_type,
        model_id=model_choice,
        log_path=log_path,
        build_dir=build_dir
    )

    results[m] = ok
    times[m] = {"total": total, "gen": gen, "err": err}

    if ok:
        generated_paths[m] = rtl_out

    print("\nTop-level result: ", "PASS" if ok else "FAIL")
    return {"generated": generated_paths, "results": results, "times": times, "skipped_top": False}


# Setting the API Key

---

In [ ]:
#@title API Key setup

# Option A (recommended): Colab secret
#   - Add a secret named: ROME-Colab
#   - Put your OpenAI API key in it
#
# Option B: environment variable (uncomment)
# os.environ["OPENAI_API_KEY"] = "sk-..."

import os
# Fetch API key ONCE and store in environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("ROME-Colab")
os.environ["MODEL"] = "ChatGPT"


# Verify the YAML + testbenches are present

---

In [ ]:
#@title Verify files in /content

from pathlib import Path
import yaml

SPEC_PATH = Path("/content/accum_module_specs.yaml")
TB_DIR    = Path("/content")
OUT_ROOT  = Path("/content/generated_accum")

assert SPEC_PATH.exists(), f"Missing YAML: {SPEC_PATH}"

tbs = sorted(TB_DIR.glob("*tb.v"))
print("Found TBs:")
for p in tbs:
    print(" -", p.name)

spec = yaml.safe_load(SPEC_PATH.read_text())
order = spec.get("module_generation_order", [m["name"] for m in spec["modules"]])

print("\nModules in YAML order (tb present?):")
for m in order:
    print(" -", m, "YES" if (TB_DIR / f"{m}tb.v").exists() else "NO")


Found TBs:
 - accum_coretb.v
 - accum_irq_gentb.v
 - accum_len_countertb.v
 - accum_rectifiertb.v
 - accum_signal_bus_iftb.v
 - accum_sra_postproctb.v
 - accum_sum_datapathtb.v
 - accum_warmup_ctrltb.v

Modules in YAML order (tb present?):
 - accum_enable_edge NO
 - accum_signal_bus_if YES
 - accum_rectifier YES
 - accum_warmup_ctrl YES
 - accum_len_counter YES
 - accum_sum_datapath YES
 - accum_sra_postproc YES
 - accum_irq_gen YES
 - accum_core YES


In [ ]:
#@title Run the hierarchical generation loop

import os
from pathlib import Path

os.environ["MODEL"] = "ChatGPT"  # or "Claude"

run = hier_gen_from_yaml(
    spec_path=Path("/content/accum_module_specs.yaml"),
    tb_dir=Path("/content"),
    out_root=Path("/content/generated_accum"),
    top_module="accum_core",
    max_iterations=10
)

print("\nRun summary:")
print("Skipped top:", run.get("skipped_top"))
print("Results:")
for k,v in run["results"].items():
    print(f" - {k}: {'PASS' if v else 'FAIL'}")

print("\nGenerated RTL files:")
for k,p in run["generated"].items():
    print(" -", k, "->", p)


Leaf modules:
 - accum_signal_bus_if -> /content/accum_signal_bus_iftb.v
 - accum_rectifier -> /content/accum_rectifiertb.v
 - accum_warmup_ctrl -> /content/accum_warmup_ctrltb.v
 - accum_len_counter -> /content/accum_len_countertb.v
 - accum_sum_datapath -> /content/accum_sum_datapathtb.v
 - accum_sra_postproc -> /content/accum_sra_postproctb.v
 - accum_irq_gen -> /content/accum_irq_gentb.v

Top module: accum_core -> /content/accum_coretb.v

[accum_signal_bus_if] iteration 1/10
Compile failed.

[accum_signal_bus_if] iteration 2/10
Simulation failed or did not pass.

[accum_signal_bus_if] iteration 3/10
Simulation failed or did not pass.

[accum_signal_bus_if] iteration 4/10
Simulation failed or did not pass.

[accum_signal_bus_if] iteration 5/10
Simulation failed or did not pass.

[accum_signal_bus_if] iteration 6/10
Simulation failed or did not pass.

[accum_signal_bus_if] iteration 7/10
Simulation failed or did not pass.

[accum_signal_bus_if] iteration 8/10
Simulation failed or did

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful